In [0]:
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np

In [0]:
cd ./drive/My Drive/WBA/gls

/content/drive/My Drive/WBA/gls


In [0]:
path_very_good = "./cabbage/extended_cabbage/very_good2"
path_good = "./cabbage/extended_cabbage/good2"
path_bad = "./cabbage/extended_cabbage/bad2"

In [0]:
import os  #必ず必要
pic_list_very_good = os.listdir(path_very_good)
pic_list_good = os.listdir(path_good)
pic_list_bad = os.listdir(path_bad)

In [0]:
%tensorflow_version 1.x
#入力画像の読み込み
from tensorflow.python.keras.preprocessing.image import load_img
import numpy as np

#ロードした画像データを格納するからのリスト
# 猫の画像をロードして、リサイズ
  
img_list_very_good = [load_img(path_very_good+"/"+name,target_size=(224,224)) for name in pic_list_very_good]
img_list_good = [load_img(path_good+"/"+name,target_size=(224,224)) for name in pic_list_good]
img_list_bad = [load_img(path_bad+"/"+name,target_size=(224,224)) for name in pic_list_bad]

In [0]:
from tensorflow.python.keras.preprocessing.image import img_to_array
def img_to_arr(img_list):
    arr_list = [img_to_array(img) for img in img_list]
    arr_list = np.array(arr_list)
    arr_list = arr_list/255
    return arr_list
arr_list_very_good = img_to_arr(img_list_very_good)
arr_list_good = img_to_arr(img_list_good)
arr_list_bad = img_to_arr(img_list_bad)

In [0]:
print(arr_list_bad.shape)
print(arr_list_good.shape)
print(arr_list_very_good.shape)

(709, 224, 224, 3)
(736, 224, 224, 3)
(705, 224, 224, 3)


In [0]:
from sklearn.model_selection import train_test_split
bad_train,bad_test= train_test_split(arr_list_bad)
good_train,good_test= train_test_split(arr_list_good)
very_good_train,very_good_test= train_test_split(arr_list_very_good)
print(bad_train.shape)
print(good_train.shape)
print(very_good_train.shape)

(531, 224, 224, 3)
(552, 224, 224, 3)
(528, 224, 224, 3)


In [0]:
x_train = np.hstack(([bad_train],[good_train],[very_good_train]))
x_train = x_train[0]
print(x_train.shape)
x_test = np.hstack(([bad_test],[good_test],[very_good_test]))
x_test = x_test[0]
print(x_test.shape)

(1611, 224, 224, 3)
(539, 224, 224, 3)


In [0]:
y_train=[[1,0,0] for i in range(bad_train.shape[0])]
for i in range(good_train.shape[0]):
    y_train.append([0,1,0])
for i in range(very_good_train.shape[0]):
    y_train.append([0,0,1])
y_train=np.array(y_train)
y_train.shape

(1611, 3)

In [0]:
y_test=[[1,0,0] for i in range(bad_test.shape[0])]
for i in range(good_test.shape[0]):
    y_test.append([0,1,0])
for i in range(very_good_test.shape[0]):
    y_test.append([0,0,1])
y_test=np.array(y_test)
y_test.shape

(539, 3)

In [0]:
from imblearn.over_sampling import RandomOverSampler

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [0]:
from tensorflow.python.keras.applications.vgg16 import VGG16
vgg16 = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg16.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 1

In [0]:
model = None
model

In [0]:
# レイヤーの調整
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Flatten

def build_transfer_model(vgg16):
    model = Sequential(vgg16.layers)
    for layer in model.layers[:15]:
        layer.trainable = False

    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='sigmoid'))

    return model

model = build_transfer_model(vgg16)



In [0]:
from tensorflow.python.keras.optimizers import SGD
model.compile(
    loss='binary_crossentropy',
    optimizer=SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy']
)
# モデルのサマリを確認
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
import math
from tensorflow.python.keras.callbacks import TensorBoard
batch_s = 32
tsb = TensorBoard(log_dir='./logs')
steps_per_epoch = math.ceil(x_train.shape[0] / batch_s)
history_model= model.fit(
  x_train,
  y_train,
  batch_size= batch_s,
  epochs=100,
  validation_split=0.2,
  callbacks=[tsb]
)

Train on 1288 samples, validate on 323 samples
Epoch 1/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.6350 - acc: 0.6788 - val_loss: 0.8038 - val_acc: 0.5377
Epoch 2/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.4368 - acc: 0.8116 - val_loss: 0.9811 - val_acc: 0.4180
Epoch 3/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.3747 - acc: 0.8468 - val_loss: 0.8700 - val_acc: 0.4727
Epoch 4/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.3320 - acc: 0.8644 - val_loss: 0.7086 - val_acc: 0.5841
Epoch 5/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.3015 - acc: 0.8786 - val_loss: 0.6623 - val_acc: 0.6006
Epoch 6/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.2796 - acc: 0.8879 - val_loss: 0.7844 - val_acc: 0.5201
Epoch 7/100
1288/1288 [==============================] - 8s 6ms/sample - loss: 0.2727 - acc: 0.8913 - val_loss: 0.6813 - val_acc: 0

In [0]:
pre_y = model.predict(x_test).argmax(axis=1)
pre_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1,

In [0]:
from sklearn.metrics import confusion_matrix
# classification_report(y_test, pre_y, target_names=["bad", "good","very_good"])
print(confusion_matrix(y_test.argmax(axis=1), pre_y))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

[[176   1   1]
 [  7 173   4]
 [  5  30 142]]
Test loss: 0.1659580293303297
Test accuracy: 0.9363018


普通のカラーのテスト精度
[[176   1   1]
 [  7 173   4]
 [  5  30 142]]
Test loss: 0.1659580293303297
Test accuracy: 0.9363018

In [0]:
model.save('./cabbage/cabbage_model.h5', include_optimizer=False)

In [0]:
pre_bad = model.predict(bad_test)
result_bad=pre_bad.argmax(axis=1)
result_bad

In [0]:
pre_good = model.predict(good_test)
result_good=pre_good.argmax(axis=1)
result_good

In [0]:
pre_very_good = model.predict(very_good_test)
result_very_good=pre_very_good.argmax(axis=1)
result_very_good

In [0]:
y_test

In [0]:
print(result_bad.shape,result_good.shape,result_very_good.shape)

In [0]:
img_list_very_good[0]